In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import DenseNet121  # Import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

# Define paths and directories
img_path = os.path.join(os.getcwd(), 'images')
base_dir = os.path.join(os.getcwd(), 'PlantVillage')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Define class directories
train_Corn____Cercospora_leaf_spot_Gray_leaf_spot_dir = os.path.join(train_dir, 'Corn____Cercospora_leaf_spot_Gray_leaf_spot')
train_Corn____Common_rust_dir = os.path.join(train_dir, 'Corn____Common_rust')
train_Corn____healthy_dir = os.path.join(train_dir, 'Corn____healthy')
train_Corn____Northern_Leaf_Blight_dir = os.path.join(train_dir, 'Corn____Northern_Leaf_Blight')

validation_Corn____Cercospora_leaf_spot_Gray_leaf_spot_dir = os.path.join(validation_dir, 'Corn____Cercospora_leaf_spot_Gray_leaf_spot')
validation_Corn____Common_rust_dir = os.path.join(validation_dir, 'Corn____Common_rust')
validation_Corn____healthy_dir = os.path.join(validation_dir, 'Corn____healthy')
validation_Corn____Northern_Leaf_Blight_dir = os.path.join(validation_dir, 'Corn____Northern_Leaf_Blight')

# Data augmentation parameters
BATCH_SIZE = 64

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True)

# Flow validation images in batches using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True)

# Create the base DenseNet-121 model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
preds = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=preds)

# Freeze layers up to a certain index (e.g., 300)
for layer in base_model.layers[:300]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
EPOCHS = 10
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    shuffle=True
)

# Save the model
model.save("densenet121_model.h5")


Found 3082 images belonging to 4 classes.
Found 770 images belonging to 4 classes.
Epoch 1/10
49/49 [==============================] - 535s 11s/step - loss: 0.2967 - accuracy: 0.8933 - val_loss: 0.2442 - val_accuracy: 0.9221
Epoch 2/10
49/49 [==============================] - 678s 14s/step - loss: 0.0931 - accuracy: 0.9620 - val_loss: 0.1134 - val_accuracy: 0.9649
Epoch 3/10
49/49 [==============================] - 323s 7s/step - loss: 0.0637 - accuracy: 0.9776 - val_loss: 0.0834 - val_accuracy: 0.9727
Epoch 4/10
49/49 [==============================] - 367s 8s/step - loss: 0.0641 - accuracy: 0.9773 - val_loss: 0.0744 - val_accuracy: 0.9740
Epoch 5/10
49/49 [==============================] - 349s 7s/step - loss: 0.0438 - accuracy: 0.9844 - val_loss: 0.0648 - val_accuracy: 0.9805
Epoch 6/10
49/49 [==============================] - 341s 7s/step - loss: 0.0322 - accuracy: 0.9883 - val_loss: 0.0651 - val_accuracy: 0.9805
Epoch 7/10
49/49 [==============================] - 343s 7s/step - lo

c:\Users\win10\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
